In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
import sys
import os
root_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
if root_path not in sys.path:
    sys.path.append(root_path)
from fileDir import getDataDir
from modules.v1.catboostModel import trainCatboost, testCatboost
from modules.v1.xgboostModel import trainXgboost, testXgboost

In [3]:
VERSION = 5
TRAIN_PATH = getDataDir("train")
TEST_PATH = getDataDir("test")
CLEANED_TRAIN_PATH = getDataDir("train", VERSION)
CLEANED_TEST_PATH = getDataDir("test", VERSION)

In [ ]:
df = pd.read_csv(TRAIN_PATH)
df.columns

## Clean Datasets

In [5]:
cat_features = [
    "Area",
    "Province",
    "gender",
    "marital_status",
    "tel_category",
    "type_of_residence",
    "c_business_type",
    "c_position",
    "c_occupation",
    "c_employment_status",
    "c_salary_payment_methods",
    "media",
    "place_for_sending_information",
    "r_propose",
    "r_generalcode1",
    "r_generalcode2",
    "r_generalcode3",
    "r_generalcode4",
    "r_generalcode5",
    "apply"
]

for col in cat_features:
    df[col] = df[col].astype(str)

In [6]:
def cleanDataset(df, cleanPath):
    # Drop unnecessary columns (keep ID)
    drop_cols = ["ID", "Shop Name"]
    df = df.drop(columns=drop_cols, errors='ignore')

    # =====================================================
    # 3. Handle date columns
    # =====================================================
    for col in ["pms_i_ymd", "date_of_birth"]:
        df[col] = pd.to_datetime(df[col], errors="coerce")
    
    df["app_year"] = df["pms_i_ymd"].dt.year
    df["app_month"] = df["pms_i_ymd"].dt.month
    df["app_day"] = df["pms_i_ymd"].dt.day
    df["app_weekday"] = df["pms_i_ymd"].dt.weekday

    df["birth_year"] = df["date_of_birth"].dt.year
    df["birth_month"] = df["date_of_birth"].dt.month
    df["birth_day"] = df["date_of_birth"].dt.day

    df["age"] = ((df["pms_i_ymd"] - df["date_of_birth"]).dt.days / 365.25).astype(float)

    df.drop(columns=["pms_i_ymd", "date_of_birth"], inplace=True, errors="ignore")

    # =====================================================
    # 4. Engineer numeric features
    # =====================================================
    df["living_period_month"] = df["living_period_year"] * 12 + df["living_period_month"]
    df.drop(columns=['living_period_year'], inplace=True)

    df["c_number_of_working_month"] = df["c_number_of_working_year"] * 12 + df["c_number_of_working_month"]
    df.drop(columns=['c_number_of_working_year'], inplace=True)

    df["r_total_income"] = df[["c_monthly_salary", "r_additional_income", "r_spouse_income"]].sum(axis=1)
    df["r_debt_to_income_ratio"] = df["r_allloan_amount"] / (df["r_total_income"] + 1)
    df["loan_to_salary_ratio"] = df["r_expected_credit_limit"] / (df["c_monthly_salary"] + 1)
    df["has_other_loans"] = (df["r_allloan_case"] > 0).astype(int)
    df["is_married_and_has_children"] = ((df["marital_status"] == 2) & (df["number_of_children"] > 0)).astype(int)

    # =====================================================
    # 5. Cleaning Data
    # =====================================================
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    df.replace(["NaN", "nan", "None", ""], np.nan, inplace=True)

    df.to_csv(cleanPath, index=False)
    return df

In [ ]:
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)
ids = test_df["ID"].copy()

train_df = cleanDataset(train_df, CLEANED_TRAIN_PATH)
test_df = cleanDataset(test_df, CLEANED_TEST_PATH)

## Testing

In [ ]:
threshold = trainXgboost(VERSION, train_df)

In [ ]:
testXgboost(VERSION, test_df, ids, threshold)

In [ ]:
threshold = trainCatboost(VERSION, train_df)

In [ ]:
testCatboost(VERSION, test_df, ids, threshold)